In [228]:
import pandas as pd
import re

In [229]:
df = pd.read_csv("food_coded.csv")

In [230]:
df.head()

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,soup,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight
0,2.4,2,1,430,NaN,315.0,1,none,we dont have comfort,9.0,...,1.0,1.0,1,1165.0,345,car racing,5,1,1315,187
1,3.654,1,1,610,3.0,420.0,2,"chocolate, chips, ice cream","Stress, bored, anger",1.0,...,1.0,1.0,2,725.0,690,Basketball,4,2,900,155
2,3.3,1,1,720,4.0,420.0,2,"frozen yogurt, pizza, fast food","stress, sadness",1.0,...,1.0,2.0,5,1165.0,500,none,5,1,900,I'm not answering this.
3,3.2,1,1,430,3.0,420.0,2,"Pizza, Mac and cheese, ice cream",Boredom,2.0,...,1.0,2.0,5,725.0,690,NaN,3,1,1315,"Not sure, 240"
4,3.5,1,1,720,2.0,420.0,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1.0,...,1.0,1.0,4,940.0,500,Softball,4,2,760,190


In [231]:
df = df.iloc[:, [52, 41, 22, 20, 15, 2, 1, 7, 33, 60]]

In [232]:
df.head()

,sports,marital_status,exercise,employment,drink,breakfast,Gender,comfort_food,healthy_feeling,weight
0,1.0,1.0,1.0,3.0,1.0,1,2,none,2,187
1,1.0,2.0,1.0,2.0,2.0,1,1,"chocolate, chips, ice cream",5,155
2,2.0,2.0,2.0,3.0,1.0,1,1,"frozen yogurt, pizza, fast food",6,I'm not answering this.
3,2.0,2.0,3.0,3.0,2.0,1,1,"Pizza, Mac and cheese, ice cream",7,"Not sure, 240"
4,1.0,1.0,1.0,2.0,2.0,1,1,"Ice cream, chocolate, chips",6,190


In [233]:
df.loc[:, "is_active"] = df.sports == 1
df.loc[:, "is_single"] = df.marital_status == 1
df.loc[:, "is_sporty"] = df.exercise == 1
df.loc[:, "is_employed"] = (df.employment == 1) | (df.employment == 2)
df.loc[:, "is_healthy_eating"] = (df.drink == 1) & (df.breakfast == 1)
df.loc[:, "gender"] = df.Gender - 1
df.loc[:, "weight"] = df.weight.fillna(0).apply(lambda x: int(int(*re.findall("[0-9]+", str(x))) / 2.2))

In [234]:
cols = ["is_sporty", "is_single", "is_active", "is_employed", "gender", "is_healthy_eating", "weight", "comfort_food", "healthy_feeling"]


In [235]:
df = df[cols]

In [236]:
categories = set()

In [237]:
for _, row in df.comfort_food.iteritems():
    try:
        categories |= {a.lower().replace("and", "").strip() for a in row.split(",")}
    except:
        pass

In [462]:
veg = ["broccoli", "carrots", "cucumber", "fruit", "grapes", "watermelon", "rice"]
sport = ["almonds", "cereal", "yogurt", "garlic", "omelet", "shake"]
ten_min = ["burger", "wings", "chips", "popcorn", "nugg", "cookie",
          "donut", "fast", "snacks"]
classic = ["beef", "grilled", "lasagn", "potato", "meat", "noodle", "peanut", "pasta", "soup", "toast",
          "burritos", "micro", "chocolate"]
premium = ["curry", "chinese", "indian", "korean", "pizza", "spaghetti", "sushi", "wine", "truffles"]
holiday = ["ice", "cake", "brownies", "macaro", "mozzarella"]

In [463]:
for _, row in df.comfort_food.iteritems():
    try:
        user_input = [a.lower().replace("and", "").strip() for a in row.split(",")]
        result = [0 for _ in range(6)]
        for i, cat_gr in enumerate([veg, sport, ten_min, classic, premium, holiday]):
            for x in user_input:
                for cat in cat_gr:
                    if cat in x:
                        result[i] = 1
        for j in range(len(result)):
            df.loc[_, f'target_{j}'] = result[j]
    except:
        for i in range(6):
            df.loc[i, f'target_{i}'] = 0

In [464]:
y.sum()

target_0    11
target_1     6
target_2    85
target_3    34
target_4    50
target_5    66
dtype: int64

In [240]:
X = df.loc[:, cols[:7]].astype(int)
y = df.loc[:, [f'target_{i}' for i in range(6)]].fillna(0).astype(int)

In [428]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier

In [403]:
from sklearn.ensemble import GradientBoostingClassifier

In [404]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [430]:
gbm_multi = OneVsRestClassifier(GradientBoostingClassifier())
pipe = Pipeline([('oversample', SMOTE()), ('gbm', gbm_multi)])

In [433]:
y_train.shape

(87, 6)

In [434]:
X_train.shape

(87, 7)

In [436]:
smote = SMOTE()

In [437]:
x1,y1=smote.fit_sample(X_train, y_train.iloc[:, 0])
x2,y2=smote.fit_sample(X_train, y_train.iloc[:, 1])
x3,y3=smote.fit_sample(X_train, y_train.iloc[:, 2])
x4,y4=smote.fit_sample(X_train, y_train.iloc[:, 3])
x5,y5=smote.fit_sample(X_train, y_train.iloc[:, 4])
x6,y6=smote.fit_sample(X_train, y_train.iloc[:, 5])

In [459]:
x1

array([[ 1.        ,  1.        ,  0.        , ...,  0.        ,
         0.        , 61.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.        , 81.        ],
       [ 0.        ,  1.        ,  0.        , ...,  1.        ,
         0.        , 93.        ],
       ...,
       [ 1.        ,  0.53640262,  0.46359738, ...,  0.        ,
         0.53640262, 68.66273967],
       [ 0.76528317,  0.        ,  1.        , ...,  0.        ,
         1.        , 61.35698219],
       [ 0.50430996,  0.49569004,  1.        , ...,  0.49569004,
         0.50430996, 73.40949086]])

In [456]:
y6.shape

(90,)

In [450]:
np.stack([y1,y2,y3,y4,y5,y6], axis=0)

ValueError: all input arrays must have the same shape

In [431]:
gs = GridSearchCV(pipe, {"gbm__estimator__max_depth": range(1, 5), "gbm__estimator__learning_rate": np.arange(0.01, 0.1, 0.02)})
gs.fit(X_train, y_train)

ValueError: bad input shape (58, 6)

In [416]:
gs.score(X_test, y_test)

0.21052631578947367

In [415]:
gs.predict_proba(X_test)

array([[0.06451966, 0.02511973, 0.61179886, 0.32219977, 0.42427073,
        0.37727388],
       [0.07996633, 0.0396039 , 0.61179886, 0.31892578, 0.43081751,
        0.54629231],
       [0.06451966, 0.02511973, 0.68964454, 0.18351395, 0.40242152,
        0.48038952],
       [0.14076895, 0.09216685, 0.75119116, 0.31407088, 0.37378192,
        0.51207525],
       [0.07996633, 0.05965308, 0.76463191, 0.29989987, 0.28783257,
        0.4798147 ],
       [0.11504602, 0.0396039 , 0.6876667 , 0.31407088, 0.40844632,
        0.44816196],
       [0.07996633, 0.02511973, 0.68964454, 0.12896302, 0.2978521 ,
        0.67977922],
       [0.11504602, 0.0396039 , 0.6876667 , 0.31407088, 0.40844632,
        0.40421791],
       [0.07996633, 0.05965308, 0.76463191, 0.40176289, 0.28273655,
        0.47633865],
       [0.07996633, 0.09216685, 0.73771565, 0.3092822 , 0.28783257,
        0.50091255],
       [0.11504602, 0.0396039 , 0.6876667 , 0.31407088, 0.40844632,
        0.44816196],
       [0.14076895, 0

сильные фичи
* 52 - спорт да(1) , нет (2, 99)
* 41 - отношения один(1), остальное(2,3,4,5,6)
* 22 - спорт да (1,2), нет (3,4,5)

слабые фичи
* 20 - работа занят(1,2), нет(3,4)
* 15 - здоровое питание да(1), нет(2)
* 3 - здоровый завтрак да(1), нет(2)
* 2 - пол ж(1), м(2)

таргет фичи
* 8 - комфортная еда
* 33 - ощущения здоровья от 1 очень здоров до 10 нездоров

In [466]:
from sklearn.externals import joblib
joblib.dump(clf, 'filename.pkl') 

TypeError: 'NoneType' object is not callable